In [1]:
from OurTrainingTools import *

In [71]:
class OurCDModel(nn.Module):
### Defines the  model with parametrized discriminant. Only quadratic dependence on a single parameter is implemented.
### Input is the architecture (list of integers, the last one being equal to 1) and the activation type ('ReLU' or 'Sigmoid')
    def __init__(self, NumberOfParameters, AR = [1, 3, 3, 1] , AF = 'ReLU'):               
        super(OurCDModel, self).__init__() 
        ValidActivationFunctions = {'ReLU': torch.relu, 'Sigmoid': torch.sigmoid}
        try:
            self.ActivationFunction = ValidActivationFunctions[AF]
        except KeyError:
            print('The activation function specified is not valid. Allowed activations are %s.'
                 %str(list(ValidActivationFunctions.keys())))
            print('Will use ReLU.')
            self.ActivationFunction = torch.relu            
        if type(AR) == list:
            if( ( all(isinstance(n, int) for n in AR)) and ( AR[-1] == 1) ):
                self.Architecture = AR
            else:
                print('Architecture should be a list of integers, the last one should be 1.')
                raise ValueError             
        else:
            print('Architecture should be a list !')
            raise ValueError
        self.NumberOfParameters = NumberOfParameters

### Define Layers
        self.NumberOfNetworks = ((2+NumberOfParameters)*(1+NumberOfParameters)/2)-1
        LinearLayers = [([nn.Linear(self.Architecture[i], self.Architecture[i+1]) \
                                  for i in range(len(self.Architecture)-1)])\
                        for n in range(self.NumberOfNetworks)]
        LinearLayers = [Layer for SubLayerList in LinearLayers for Layer in SubLayerList]
        self.LinearLayers = nn.ModuleList(LinearLayers)
        
    def Forward(self, Data, Parameters):
### Forward Function. Performs Preprocessing, returns F = rho/(1+rho) in [0,1], where rho is quadratically parametrized.
        # Checking that data has the right input dimension
        InputDimension = self.Architecture[0]
        if Data.size(1) != InputDimension:
            print('Dimensions of the data and the network input mismatch: data: %d, model: %d'
                  %(Data.size(1), InputDimension))
            raise ValueError

        # Checking that preprocess has been initialised
        if not hasattr(self, 'Shift'):
            print('Please initialize preprocess parameters!')
            raise ValueError
        with torch.no_grad(): 
            Data, Parameters = self.Preprocess(Data, Parameters)  
        
        NumberOfLayers, NumberOfEvents = len(self.Architecture)-1, Data.size(0)
        EntryIterator, NetworkIterator = 0, -1
        MatrixLT = torch.zeros([NumberOfEvents, (self.NumberOfParameters+1)**2], dtype=Data.dtype)
        
        if Data.is_cuda:
            MatrixLT = OurCudaTensor(MatrixLT)
        
        for i in range(self.NumberOfParameters+1):
            EntryIterator += i
            for j in range(self.NumberOfParameters+1-i):
                if NetworkIterator == -1:
                    MatrixLT[:, EntryIterator] = torch.ones(NumberOfEvents)
                    print('Entry: %d, Layer: ones'%(EntryIterator))
                else:
                    x = Data
                    for Layer in self.LinearLayers[NumberOfLayers*NetworkIterator:\
                                                  NumberOfLayers*(NetworkIterator+1)-1]:
                        x = self.ActivationFunction(Layer(x))
                    x = self.LinearLayers[NumberOfLayers*(NetworkIterator+1)-1](x).squeeze()
                    MatrixLT[:, EntryIterator] = x
                    print('Entry: %d, Layer: %d'%(EntryIterator, NetworkIterator))
                EntryIterator += 1
                NetworkIterator += 1
        #print('MatrixLT: '+str(MatrixLT.is_cuda))
        #print('Parameters: '+str(Parameters.is_cuda))

        MatrixLT = MatrixLT.reshape([-1, self.NumberOfParameters+1, self.NumberOfParameters+1])
        MatrixLTP = MatrixLT.matmul(Parameters.reshape([NumberOfEvents, self.NumberOfParameters+1, 1]))
        rho = MatrixLTP.permute([0, 2, 1]).matmul(MatrixLTP).squeeze()
        
        return (rho.div(1.+rho)).view(-1, 1)
    
    def GetL1Bound(self, L1perUnit):
        self.L1perUnit = L1perUnit
    
    def ClipL1Norm(self):
### Clip the weights      
        def ClipL1NormLayer(DesignatedL1Max, Layer, Counter):
            if Counter == 1:
                ### this avoids clipping the first layer
                return
            L1 = Layer.weight.abs().sum()
            Layer.weight.masked_scatter_(L1 > DesignatedL1Max, 
                                        Layer.weight*(DesignatedL1Max/L1))
            return
        
        Counter = 0
        for m in self.children():
            if isinstance(m, nn.Linear):
                Counter += 1
                with torch.no_grad():
                    DesignatedL1Max = m.weight.size(0)*m.weight.size(1)*self.L1perUnit
                    ClipL1NormLayer(DesignatedL1Max, m, Counter)
            else:
                for mm in m:
                    Counter +=1
                    with torch.no_grad():
                        DesignatedL1Max = mm.weight.size(0)*m.weight.size(1)*self.L1perUnit
                        ClipL1NormLayer(DesignatedL1Max, mm, Counter)
        return 
    
    def DistributionRatio(self, points):
### This is rho. I.e., after training, the estimator of the distribution ratio.
        with torch.no_grad():
            F = self(points)
        return F/(1-F)

    def InitPreprocess(self, Data, Parameters):
### This can be run only ONCE to initialize the preprocess (shift and scaling) parameters
### Takes as input the training Data and the training Parameters as Torch tensors.
        if not hasattr(self, 'Scaling'):
            print('Initializing Preprocesses Variables')
            self.Scaling = Data.std(0)
            self.Shift = Data.mean(0)
            self.ParameterScaling = Parameters.std(0)  
        else: print('Preprocess can be initialized only once. Parameters unchanged.')
            
    def Preprocess(self, Data, Parameters):
### Returns scaled/shifted data and parameters
### Takes as input Data and Parameters as Torch tensors.
        if  not hasattr(self, 'Scaling'): print('Preprocess parameters are not initialized.')
        Data = (Data - self.Shift)/self.Scaling
        Parameters = Parameters/self.ParameterScaling
        Ones = torch.ones([Parameters.size(0),1], dtype=Parameters.dtype)
        if Parameters.is_cuda:
            Ones = Ones.cuda()
        Parameters = torch.cat([Ones, Parameters.reshape(-1,1)], dim=1)
        return Data, Parameters
    
    def Save(self, Name, Folder, csvFormat=False):
### Saves the model in Folder/Name
        FileName = Folder + Name + '.pth'
        torch.save({'StateDict': self.state_dict(), 
                   'Scaling': self.Scaling,
                   'Shift': self.Shift,
                   'ParameterScaling': self.ParameterScaling}, 
                   FileName)
        print('Model successfully saved.')
        print('Path: %s'%str(FileName))
        
        if csvFormat:
            modelparams = [w.detach().tolist() for w in self.parameters()]
            np.savetxt(Folder + Name + ' (StateDict).csv', modelparams, '%s')
            statistics = [self.Shift.detach().tolist(), self.Scaling.detach().tolist(),
                         self.ParameterScaling.detach().tolist()]
            np.savetxt(Folder + Name + ' (Statistics).csv', statistics, '%s')
    
    def Load(self, Name, Folder):
### Loads the model from Folder/Name
        FileName = Folder + Name + '.pth'
        try:
            IncompatibleKeys = self.load_state_dict(torch.load(FileName)['StateDict'])
        except KeyError:
            print('No state dictionary saved. Loading model failed.')
            return 
        
        if list(IncompatibleKeys)[0]:
            print('Missing Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        if list(IncompatibleKeys)[1]:
            print('Unexpected Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        self.Scaling = torch.load(FileName)['Scaling']
        self.Shift = torch.load(FileName)['Shift']
        self.ParameterScaling = torch.load(FileName)['ParameterScaling']
        
        print('Model successfully loaded.')
        print('Path: %s'%str(FileName))
        
    def Report(self): ### is it possibe to check if the model is in double?
        print('\nModel Report:')
        print('Preprocess Initialized: ' + str(hasattr(self, 'Shift')))
        print('Architecture: ' + str(self.Architecture))
        print('Loss Function: ' + 'Quadratic')
        print('Activation: ' + str(self.ActivationFunction))
        
    def cuda(self):
        nn.Module.cuda(self)
        self.Shift = self.Shift.cuda()
        self.Scaling = self.Scaling.cuda()
        self.ParameterScaling = self.ParameterScaling.cuda()
        
    def cpu(self):
        self.Shift = self.Shift.cpu()
        self.Scaling = self.Scaling.cpu()
        self.ParameterScaling = self.ParameterScaling.cpu()
        return nn.Module.cpu(self)

    



In [ ]:
td = OurTrainingData(['/data3/Training/TrainingData/SMData/ChPsm.h5',
                     '/data3/Training/TrainingData/SMData/ChPsm2.h5',
                     '/data3/Training/TrainingData/SMData/ChPsm3.h5',],
                    ['/data3/Training/TrainingData/GWData/ChPgw2.h5',
                     '/data3/Training/TrainingData/GWData/ChPgw5.h5',
                     '/data3/Training/TrainingData/GWData/ChPgw10.h5',
                     '/data3/Training/TrainingData/GWData/ChPgwm2.h5',
                     '/data3/Training/TrainingData/GWData/ChPgwm5.h5',
                     '/data3/Training/TrainingData/GWData/ChPgwm10.h5',],
                     process = 'W+Z', parameters =['GW[TeV**-2]'], 
                     SMNLimits=int(4e2),
                     BSMNLimits=int(2e2))

Data, ParVal, Labels, Weights = td.Data[:, :8], td.ParVal, td.Labels, td.Weights
MD = OurCDModel(NumberOfParameters=1, AR=[8,32,32,32,1])
MD.double()
MD.InitPreprocess(Data, ParVal)
OT = OurTrainer(NumEpochs = int(1e4))
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))

OT.Train(MD, Data = Data, Parameters = ParVal, Labels=Labels, Weights= Weights, bs = 100000,
        Name = 'ChPgw2,5,10, (200k, Cholesky Decomposition), ', Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W+Z, with new physics Parameters: ['GW[TeV**-2]']

Reading file .../data3/Training/TrainingData/GWData/ChPgw2.h5
##### File Info:
GW = 0.02[Tev**-2] data, showering on. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ, PtWZ, PtW, mz2, mw2}, weight}. Converted from /data3/ConvertMGevents/Showered_Events_for_Training/ChPLong290gw2.dat
 File with charge = 1
#####

Reading file .../data3/Training/TrainingData/GWData/ChPgw5.h5
##### File Info:
GW = 0.05[Tev**-2] data, showering on. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ, PtWZ, PtW, mz2, mw2}, weight}. Converted from /data3/ConvertMGevents/Showered_Events_for_Training/ChPLong290gw5.dat
 File with charge = 1
#####

Reading file .../data3/Training/TrainingData/GWData/ChPgw10.h5
##### File Info:
GW = 0.1[Tev**-2] data, showering on. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ, PtWZ, PtW, mz2, mw2}, weight}. Converted from /data3/ConvertMGevents/Showered_Events_for_Training/ChPLong290gw10.dat
 File 

In [56]:
ParVal.is_cuda

False

In [45]:
ParVal.size()

torch.Size([4800000])

In [4]:
TempParameters.size(), (MatrixLT).size()

(torch.Size([100, 3]), torch.Size([100, 3, 3]))

In [19]:
TempParameters.reshape([100, 1, 3]).matmul(MatrixLT.permute([0, 2, 1,]))[0]

tensor([[ 2.0776,  6.5665, -2.4170]], grad_fn=<SelectBackward>)

In [25]:
MatrixLT[0].transpose(0, 1)

tensor([[ 1.0000,  0.0000,  0.0000],
        [-0.1251,  0.1466,  0.0000],
        [ 0.0266,  0.1255, -0.0483]], grad_fn=<TransposeBackward0>)

In [27]:
TempParameters[0].reshape([1, 3]).mm(MatrixLT[0].transpose(0, 1))

tensor([[ 2.0776,  6.5665, -2.4170]], grad_fn=<MmBackward>)

In [34]:
MatrixLTP = MatrixLT.matmul(TempParameters.reshape([100, 3, 1]))
MatrixLTP.permute([0, 2, 1]).matmul(MatrixLTP).squeeze()

tensor([53.2776, 41.2998, 39.7028, 60.2778, 37.5379, 50.8842, 40.7845, 41.8562,
        41.7262, 57.7348, 62.5857, 64.4010, 40.5552, 36.0387, 41.7204, 44.0750,
        41.8498, 55.1948, 57.6788, 33.6665, 38.0422, 78.1223, 34.6115, 55.5912,
        52.7553, 29.8227, 45.6006, 56.3874, 29.8103, 46.8122, 55.6364, 38.1525,
        47.8337, 40.9304, 45.7424, 25.9865, 48.1553, 60.4051, 72.2461, 59.1649,
        30.3844, 46.5015, 40.5201, 40.7097, 55.5965, 41.4380, 54.1305, 43.2585,
        46.9714, 39.4395, 62.3753, 74.8668, 54.3242, 34.5350, 47.0982, 48.9572,
        41.5068, 39.7230, 44.4457, 53.8537, 73.3045, 55.2483, 40.0775, 39.9372,
        49.8916, 48.3826, 34.7451, 23.2622, 54.1850, 40.4493, 48.1900, 54.7582,
        49.6336, 40.2133, 49.5574, 36.5985, 46.7742, 59.3095, 41.3831, 42.0143,
        45.2201, 38.8211, 69.8754, 37.5511, 47.5282, 41.5546, 43.2678, 59.7412,
        25.9005, 38.3847, 63.8126, 33.0527, 52.6752, 63.7158, 58.9484, 36.8365,
        49.9637, 33.6771, 63.9414, 52.88